In [151]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import RegexpTokenizer

%matplotlib inline

nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\tclem\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tclem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tclem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [152]:
#import re
#from unidecode import unidecode
#import unicodedata as ud

RESULTS_CSV = '..\data\stanford\inn.csv'
data = pd.read_csv(RESULTS_CSV).replace(np.nan, "", regex = True)

data.head()

,commentthreadid,forumpostid,forumuid,staffcandidate,staff,nonstaff,Sentiment,Confusion,Urgency,Opinion,...,Text,upcount,reads,CourseType,coursedisplayname,posttype,anonymous,anonymoustopeers,opinions,demandreviewreason
0,51b79a60f359c412000000c7,51d3a427bbdc2e0a0000004c,050AF8325C245EDC922ED2BDA93C84F0,False,False,False,4.0,4.0,2.0,False,...,Follow the link below to see a function you can copy and paste into your R console at the > prompt. Press enter to create the function. Type ls() and press enter to see if it's there. Then type dot.plot(rnorm(100)) and press enter again to see a dot plot.http://www.stats.waikato.ac.nz/Courses/stat221a_03/r_files/dot.plot.rLooking at the code might help you learn R. Hope that helps.,0,137,Medicine,Medicine/HRP258/Statistics_in_Medicine,Comment,False,False,"{51b7eb0ff359c40e0000011f=0.8620117902755737, 51b8c64f4983c00a0000006e=0.007799714803695679}",first opinion is not the first comment / more than one opinion
1,51b79a60f359c412000000c7,51b990b54983c010000000db,423087C3AC2011004E642BA6FBA57AAA,False,False,True,4.5,4.0,2.5,False,...,It seems to be interesting. I will try the R homeworks when i will finish with the lectures!,0,137,Medicine,Medicine/HRP258/Statistics_in_Medicine,Comment,False,False,"{51b7eb0ff359c40e0000011f=0.7811994155248007, 51b8c64f4983c00a0000006e=0.08967032531897226}",first opinion is not the first comment / more than one opinion
2,51b79a60f359c412000000c7,51b907ef4983c00e000000a6,423087C3AC2011004E642BA6FBA57AAA,False,False,True,6.0,4.0,1.5,False,...,Kristen is amazing...Thank you!!,0,137,Medicine,Medicine/HRP258/Statistics_in_Medicine,Comment,False,False,"{51b7eb0ff359c40e0000011f=0.45011393229166663, 51b8c64f4983c00a0000006e=0.333575040102005}",first opinion is not the first comment / more than one opinion
3,51b79a60f359c412000000c7,51b8c64f4983c00a0000006e,65A1D8A2220ABD82B005C4A06ED32213,False,False,True,4.0,4.0,3.0,True,...,The R deducer software seems to have quite a useful set of plots available,1,137,Medicine,Medicine/HRP258/Statistics_in_Medicine,Comment,False,False,"{51b7eb0ff359c40e0000011f=0.7992700338363647, 51b8c64f4983c00a0000006e=1.0}",first opinion is not the first comment / more than one opinion
4,51b79a60f359c412000000c7,51b7eb0ff359c40e0000011f,5DC812268219A1754C34D367E342C0A8,False,False,False,4.0,4.0,2.5,True,...,GeoGebra also has all kinds of plot functionality. It is free and online.,1,137,Medicine,Medicine/HRP258/Statistics_in_Medicine,Comment,False,False,"{51b7eb0ff359c40e0000011f=1.0, 51b8c64f4983c00a0000006e=0.007371455430984497}",first opinion is not the first comment / more than one opinion


In [149]:
#data.sort_values(by=['forumuid'])
import statistics
#data.insert(0, 'mean_Urgency',None)
#data.insert(0, 'postings',None)
grouped = data.groupby('commentthreadid')
def vai(data):
    data.loc[:,'mean_Urgency'] = statistics.mean(pd.to_numeric(data.loc[:,'Urgency'].str.replace(',','.')))
postingVectors_writer=pd.DataFrame()
for group_name, d in grouped:
    d.loc[:,'mean_Urgency'] = statistics.mean(pd.to_numeric(d.loc[:,'Urgency'].replace(',','.')))
    d.loc[:,'postings'] = len(d)
    
    if(postingVectors_writer.empty):
        postingVectors_writer = d.copy(deep=False)
    else:
        postingVectors_writer = postingVectors_writer.append(d)
    #d.loc[:,'mean_Urgency'] = statistics.mean(pd.to_numeric(d.loc[:,'Urgency'].str.replace(',','.')))

In [150]:
postingVectors_writer.loc[postingVectors_writer['postings'] >= 3].sort_values(by=['mean_Urgency','commentthreadid','forumpostid'], ascending=False).head(130)

,postings,mean_Urgency,commentthreadid,forumpostid,forumuid,staffcandidate,staff,nonstaff,Sentiment,Confusion,...,Text,upcount,reads,CourseType,coursedisplayname,posttype,anonymous,anonymoustopeers,opinions,demandreviewreason
7921,3,6.166667,53c145c3c51b37e1e7000025,53c22f9255ffd7056a000004,A259E68792F546EBA9CAD5CC6B51E25C,False,False,False,4.0,5.0,...,"I also have the same problem. Would the technical guys look into this, please. My browser is Safari on Macbook.",1,48,Humanities,HumanitiesSciences/Econ-1/Summer2014,Comment,False,False,{53c18ebbc51b37e1e7000035=0.08360726634661356},first opinion is not the first comment
7922,3,6.166667,53c145c3c51b37e1e7000025,53c18ebbc51b37e1e7000035,0E8D2B616AF23853476BA23275631FF4,False,False,False,3.5,5.0,...,Hii have the same problem,1,48,Humanities,HumanitiesSciences/Econ-1/Summer2014,Comment,False,False,{53c18ebbc51b37e1e7000035=1.0},first opinion is not the first comment
7923,3,6.166667,53c145c3c51b37e1e7000025,53c145c3c51b37e1e7000025,AFE967B62578DA61397807AB33C2FA6B,False,False,False,3.5,5.0,...,"Hi, my name is <nameRedac_<anon_screen_name_redacted>> <nameRedac_<anon_screen_name_redacted>> <nameRedac_<anon_screen_name_redacted>> and I am with a problem in my progress tab!I have had problems with the quizzes for deviations from competition'''', that is, the quiz 10 is graded but not posted to progress tab! The credits I have done for the quizzes in to three subsections ''''Monopoly and Market power'''', ''''Between Monopoly and Competition'''' and ''''Antitrust Policy and Regulation'''' on Deviations from Competition'''' in Principles of Economics course, is not shown in my progress tab. Take a look at the forum discussions to see claims of other students about this same problem. I Hope this is going to be fixed soon! It`s urgent for me!Solve this problem as quickly as possible please!I need an answer urgently!my email: <redacted>''''",3,48,Humanities,HumanitiesSciences/Econ-1/Summer2014,CommentThread,False,False,{53c18ebbc51b37e1e7000035=3.482401371002197E-4},first opinion is not the first comment
7619,3,6.000000,53aaaff4f2f1b7fb240000ae,53aed2ecad7d7cff2b000014,0F32DBE3313D89F43A53415EE17C610E,False,False,False,4.0,5.0,...,Problem needs to be fixed.<nameRedac_<anon_screen_name_redacted>>,0,162,Humanities,HumanitiesSciences/Econ-1/Summer2014,Comment,False,False,"{53aaaff4f2f1b7fb240000ae=0.08402134478092194, 53ab3f4dd2efd3bb14000034=0.11761614680290222, 53aed2ecad7d7cff2b000014=1.0}",more than one opinion / more than one opinion
7620,3,6.000000,53aaaff4f2f1b7fb240000ae,53ab3f4dd2efd3bb14000034,0B806A6E0A24BE184271CF53E1F91FB0,False,False,False,4.0,5.0,...,"Also, when you changed the Q3 second question from .38 to .34, you should have reset all the submission. When I posted my answer both were .38!",1,162,Humanities,HumanitiesSciences/Econ-1/Summer2014,Comment,False,False,"{53aaaff4f2f1b7fb240000ae=0.11929986874262491, 53ab3f4dd2efd3bb14000034=1.0, 53aed2ecad7d7cff2b000014=0.003977566957473755}",more than one opinion / more than one opinion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5449,4,4.750000,5267ba5bad8969aa8800001a,526967edc35d0262af00000c,0FA78DE09E838CBB5DADAD13460046C7,False,True,False,4.0,4.0,...,"We have temporary access to TurnItIn software, which a lot of universities are using. It is quite good--as it shows you exactly where material has been copied from on the internet. So, yes, there's great software out there!",0,115,Medicine,Medicine/SciWrite/Fall2013,Comment,False,False,{526cf3555e65538ad2000012=0.10292011499404907},first opinion is not the first comment
5450,4,4.750000,5267ba5bad8969aa8800001a,5267ba5bad8969aa8800001a,E360B0CBAA46345269F0B79DC3C38B09,False,False,False,4.0,4.5,...,Hi CristinDo you recommend the use of plagiarism-detecting software as many journals do?Thanks,0,115,Medicine,Medicine/SciWrite/Fall2013,CommentThread,False,False,{526cf3555e65538ad2000012=0.030950486660003662},first opinion is not th

In [78]:
pd.set_option('display.max_colwidth', -1)
postingVectors_writer.loc[postingVectors_writer['postings'] >= 3].sort_values(by=['mean_Urgency','forumuid','forumpostid'], ascending=False)[7:10].Text

22733    I really enjoyed this course and congratulate Professor <redacted> on a job well done.  Were the *starting* point assessment and *pre*-course survey always there and I missed them, or did they just show up?   Are we going to get the answers to the assessment just for curiosity's sake?                                                                                                          
20402    When calculating the normal approximation for the binomial do we use half intervals e.g. 111.5 instead of 112 if we're looking for the area to the right in the curve: and can we use a different normal distribution applet than the ones recommended. If so, what are the answer tolerances. Can one be several tenths of a percentage point off. This ambiguity is killing me. Literally, I'm dying.
19659    How many binary variables?  Does this mean the lines that can be designated binary or the individual entries of data? In other words x or 2x?                                                

In [ ]:
import matplotlib
from matplotlib.ticker import FuncFormatter

def to_percent(y, position):
    s = str(round(100 * (y/3)))
    if matplotlib.rcParams['text.usetex'] is True:
        return s + r'$\%$'
    else:
        return s + '%'
    
formatter = FuncFormatter(to_percent)

x_labels = ['Não-opinião','Opinião']

fig = plt.Figure(figsize=(10,5))

fig.suptitle('Urgência por Opinião', fontsize =20)
plt.ylabel('Urgência Máxima', fontsize =14)
#plt.xlabel('Opinião', fontsize =14)
plt.xticks([0,1],x_labels,fontsize=12)
plt.tick_params(labelbottom = 'off')
plt.gca().yaxis.set_major_formatter(formatter)
plt.hist([v for i,v in enumerate(postingVectors.loc[postingVectors['Urgency'].str.replace(',','.',regex=False).astype(np.float) >= 7]['Opinion'])], bins = 3, cumulative = False,density=True, color='gray', align='mid')

plt.show()

In [153]:
postings = pd.DataFrame(data={'Opinion': data['Opinion'], 
                              'Question': data['Question'], 
                              'Answer': data['Answer'], 
                              'Sentiment': data['Sentiment'], 
                              'Confusion': data['Confusion'], 
                              'Urgency': data['Urgency'], 
                              'Text': data['Text'], 
                              'forumpostid': data['forumpostid'], 
                              'CourseType': data['CourseType'], 
                              'coursedisplayname': data['coursedisplayname'], 
                              'commentthreadid': [row['forumpostid'] if row['commentthreadid'] == 'None' or row['commentthreadid'] == '' else row['commentthreadid'] for index, row in data.iterrows()]})

postings.set_index(['commentthreadid'],inplace=True,drop=True)
postings.sort_values(by=['commentthreadid'],inplace=True)

postings_aux = postings.groupby('commentthreadid')

discussions = pd.DataFrame(data={'discussion': postings_aux['Text'].sum(), 'CourseType': postings_aux['CourseType'].first(), 'coursedisplayname': postings_aux['coursedisplayname'].first()})#, 'commentthreadid': data_cleanned.groupby('commentthreadid').first().index}) 

#data_cleanned.reset_index(inplace=True)
#data_cleanned.drop(['index'], axis=1, inplace=True)
#print(re.sub(rb'[^\x00-\x7f]',rb' ',discussions.loc['51b513008e8d330d00000001','discussion'].encode('utf-8')))

#print(re.sub(r'[^\x00-\x7f]',r' ',ud.normalize('NFD',discussions.loc['51b513008e8d330d00000001','discussion'])))

#print(discussions.loc['51b513008e8d330d00000001','discussion']) 
postings['Text'].head()

commentthreadid
51b513008e8d330d00000001    Dear Madam.,   Greeting, Course MS Power Point file could not read my system (Window- MS office 03-10). Kindly resolve my problem RegardsELA                                                                                                                                                                                                                                           
51b513008e8d330d00000001    Hello jacalcache,Do you know where can i find the course Writing in the Sciences? Will be offered again?Thank you.E.A                                                                                                                                                                                                                                                                    
51b513008e8d330d00000001    Hello Kristin,I attended your last two courses (Writing in the Sciences) and both are wonderful. Thanks for this information. I woul

In [154]:
postings.reset_index(inplace=False)['Text'].head()

0    Dear Madam.,   Greeting, Course MS Power Point file could not read my system (Window- MS office 03-10). Kindly resolve my problem RegardsELA                                                                                                                                                                                                                                           
1    Hello jacalcache,Do you know where can i find the course Writing in the Sciences? Will be offered again?Thank you.E.A                                                                                                                                                                                                                                                                    
2    Hello Kristin,I attended your last two courses (Writing in the Sciences) and both are wonderful. Thanks for this information. I would like to learn from you how to teach stats to med students,<nameRedac_<anon_screen

## Tokenização

Inicialmente, foram definidas as estratégias de tokenização que seriam adotadas para, a partir daí, gerar os tokens que serão utilizados nas análises. A primeira estratégia adotada foi a **remoção das _stopwords_**. Essa remoção foi realizada através da coleção de palavras disponibilizada pela biblioteca NLTK e é motivada pelo fato que, apesar de sua frequência elevada, as _stopwords_ são pouco significantes para os textos.

As palavras adotadas como *tokens* foram mantidas respeitando a **bicameralidade** (caixa alta ou baixa), visando preservar as possíveis diferenças sintáticas. Também foram mantidas todas as palavras com **duas ou mais letras** e considerou-se que **hifens** e **apóstrofos** são caracteres integrantes das palavras.

Também foram utilizados, como *tokens*, os números inteiros e decimais com, pelo menos, **dois dígitos** e datas no formato **dd/mm/yy** ou **dd/mm/yyyy**, dada a importância que esses valores podem ter para o texto, ainda que uma possível vetorização possa vir a descartá-los.


In [155]:
regex_patterns = { "words": '''\w+[-']*\w+''',
                  "discrete": '''\d{2,}''',
                  "dates": '''\d{2}\/\d{2}\/\d{2,4}''',
                  "continuous": '''\d*[\.]*\d+[\.|\,]*\d+''' }

regex = '|'.join(regex_patterns.values())

In [156]:
def tokenize_data(data, regex):
    
    token_list = []
    token_bag = []
    tokenizer = RegexpTokenizer(regex)
    stopwords_en = stopwords.words("english")
  
    for row in data:
        
        tokens = tokenizer.tokenize(row)
        tokens = [token for token in tokens if token not in stopwords_en]
        token_bag.extend(tokens)
        token_list.append(tokens)
      
    return token_list, token_bag

postings['Text'], corpus_vocabulary = tokenize_data(postings['Text'], regex)
corpus_vocabulary_set = set(corpus_vocabulary)

In [157]:
postings.head()

,Opinion,Question,Answer,Sentiment,Confusion,Urgency,Text,forumpostid,CourseType,coursedisplayname
commentthreadid,,,,,,,,,,
51b513008e8d330d00000001,False,True,False,4.0,4.0,5.0,"[Dear, Madam, Greeting, Course, MS, Power, Point, file, could, read, system, Window, MS, office, 03-10, Kindly, resolve, problem, Regards, ELA]",51c820d83927211600000021,Medicine,Medicine/HRP258/Statistics_in_Medicine
51b513008e8d330d00000001,False,True,False,4.5,4.0,5.0,"[Hello, jacalcache, Do, know, find, course, Writing, Sciences, Will, offered, Thank]",51bef670bf600b0a00000080,Medicine,Medicine/HRP258/Statistics_in_Medicine
51b513008e8d330d00000001,False,False,False,6.0,4.0,1.5,"[Hello, Kristin, attended, last, two, courses, Writing, Sciences, wonderful, Thanks, information, would, like, learn, teach, stats, med, students, nameRedac_, anon_screen_name_redacted, nameRedac_, anon_screen_name_redacted, king, regards]",51be609bbf600b1200000052,Medicine,Medicine/HRP258/Statistics_in_Medicine
51b513008e8d330d00000001,False,False,False,6.0,4.0,1.0,"[great, course, Thanks]",51be54fbbf600b120000004f,Medicine,Medicine/HRP258/Statistics_in_Medicine
51b513008e8d330d00000001,False,True,False,3.5,4.0,5.5,"[noticed, weekly, progress, quiz, section, Unit, module, questions, However, saw, got, one, wrong, So, never, saw, two, questions, pressed, check, button, It, never, occurred, check, answer, would, every, sheet, rather, end, Can, help, Resubmit, maybe, Thanks]",51be17dbbf600b0a00000032,Medicine,Medicine/HRP258/Statistics_in_Medicine


Conhecendo os *tokens* e o vocabulário do *corpus* é possível calcular as métricas restantes e necessárias para analisar a frequência das palavras no *corpus*. Nesse processo será produzida um *Series* com as frequências das palavras que será reutilizado e complementado posteriormente.

In [158]:
def get_frequency_series(tokens):
  token_frequency = pd.Series(tokens).value_counts().reset_index()
  token_frequency.columns = ["Word", "Frequency"]
  return token_frequency

total_word_occurrences = len(corpus_vocabulary)
words_frequency = get_frequency_series(corpus_vocabulary)

Agora que as características do *corpus* utilizado estão bem definidas, há interesse em conhecer como se comporta a distribuição de frequência das palavras nele contidas. Para isso, o *series* criado anteriormente descrevendo a frequência de todas as palavras do vocabulário será complementado com as novas variáveis de interesse. Além da frequência absoluta de cada palavra, serão também analisados:

- **r:** O *rank* das palavras em relação à sua frequência
- **Pr:** A probabilidade de ocorrência da das palavras
- **r.Pr:** Os resultados da Lei de Zipf 

In [159]:
words_frequency["Ranking (r)"] = words_frequency["Frequency"].rank(ascending=False, method='first')
pr = (words_frequency["Frequency"] / total_word_occurrences)
r_pr = (words_frequency["Ranking (r)"] * pr)

Para melhor compreensão do leitor, são realizados alguns ajustes de apresentação para os valores calculados.

In [160]:
words_frequency["Ranking (r)"] = words_frequency["Ranking (r)"].astype(int)
words_frequency["Pr (%)"] = round(pr * 100, 2)
words_frequency["r.Pr"] = round(r_pr, 3)

Por fim, devido à grande quantidade de palavras no vocabulário do *corpus*, a tabela abaixo descreve apenas os resultados obtidos para as 50 palavras mais frequentes da coleção de documentos.

In [161]:
df_words_frequency = pd.DataFrame(words_frequency)
df_words_frequency.head(50)

,Word,Frequency,Ranking (r),Pr (%),r.Pr
0,students,1827,1,0.66,0.007
1,would,1808,2,0.65,0.013
2,The,1721,3,0.62,0.019
3,zipRedac,1706,4,0.62,0.025
4,course,1513,5,0.55,0.027
5,think,1497,6,0.54,0.032
6,one,1431,7,0.52,0.036
7,like,1137,8,0.41,0.033
8,get,1083,9,0.39,0.035
9,also,1071,10,0.39,0.039


In [162]:
postings.reset_index().to_json(r'..\data\stanford\postings.json')
discussions.to_json(r'..\data\stanford\discussions.json')